<a href="https://colab.research.google.com/github/av-gav/itmo-ml/blob/main/Task2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Putting it all together (TensorFlow)

In [ ]:
!pip install datasets evaluate transformers[sentencepiece] accelerate pycountry

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip drive/MyDrive/train/lid.zip

In [3]:
import pycountry
import datasets

wili = datasets.load_dataset('wili_2018')
# names for languages not in iso-639-3 from wikipedia
non_iso_languages = {'roa-tara': 'Tarantino', 'zh-yue': 'Cantonese', 'map-bms': 'Banyumasan',
                      'nds-nl': 'Dutch Low Saxon', 'be-tarask': 'Belarusian-Tarask'}

diff = {'Russia Buriat':'Buriat', 'Kirghiz':'Kyrgyz', 'Ossetian':'Ossetic'}
# create dictionary from data set labels to language names
lab2lang = {}
for i, lang in enumerate(wili['train'].features['label'].names):
  full_lang = pycountry.languages.get(alpha_3=lang)
  if full_lang:
    lab2lang[i] = diff.get(full_lang.name, full_lang.name)
  else:
    lab2lang[i] = non_iso_languages[lang]

Generating train split:   0%|          | 0/117500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/117500 [00:00<?, ? examples/s]

Dataset wili_2018 downloaded and prepared to /root/.cache/huggingface/datasets/wili_2018/WiLI-2018 dataset/1.1.0/78d7fe4a9d0a01168e45657f302c776ee0afc0978d44e2c3759f4c4975b845f5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [41]:
from datasets import ClassLabel
ds = datasets.load_from_disk('lid')

features = ds['train'].features.copy()
class_label = features['label']
langs = class_label.names
id2label = {idx: langs[idx] for idx in range(len(langs))}
label2id = {v: k for k,v in id2label.items()}
common = [label2id[x] for x in langs if x in lab2lang.values()]
features['label'] = ClassLabel(names=[lab2lang[k] for k in sorted(lab2lang)])
mapper = {label2id[v]:k for k,v in lab2lang.items() if v in label2id}
def map_lang(x):
  x['label'] = mapper[x['label']]
  return x
ds_part = ds.filter(lambda x: x['label'] in common).cast(features).map(map_lang)

Map:   0%|          | 0/38276 [00:00<?, ? examples/s]

Map:   0%|          | 0/9568 [00:00<?, ? examples/s]

In [49]:
n_labs = len(lab2lang)
id2lab = {i:f'LAB_{i}' for i in range(n_classes)}
lab2id = {v:k for k,v in id2lab_n.items()}

In [51]:
from transformers import CanineTokenizer, AutoModelForSequenceClassification, pipeline

tokenizer = CanineTokenizer.from_pretrained("google/canine-c", padding="longest")

model = AutoModelForSequenceClassification.from_pretrained("SebOchs/canine-c-lang-id",
                                                             num_labels=n_labs, id2label=id2lab, label2id=lab2id)

pipe = pipeline("text-classification",
                tokenizer = tokenizer,
                model = model,
                devoce = 0)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
import evaluate

metric = evaluate.load('accuracy')
evaluator = evaluate.evaluator("text-classification")

result = evaluator.compute(model_or_pipeline=pipe, 
                           data=ds_part['test'].select(range(1000)), metric=metric, label_mapping=lab2id)

print(result)
